In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece
! pip install spacy==2.0.12
! pip install bert-extractive-summarizer
!pip install -q gradio

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 6.5 MB 29.5 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 596 kB 41.3 MB/s 
     |████████████████████████████████| 895 kB 40.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.4 MB/s 
     |████████████████████████████████| 22.0 MB 66.9 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 1.2 MB 46.2 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 150 kB 46.9 MB/s 
     |████████████████████████████████| 601 kB 50.6 MB/s 
     |████████████████████████████████| 243 kB 44.9 MB/s 
     |████████████████████████████████| 443 kB 42.2 MB/s 
  Created wheel for spacy: filename=spacy-2.0.

In [3]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from summarizer import Summarizer,TransformerSummarizer
import torch
import torchvision.models as models
import warnings
warnings.filterwarnings('ignore')
import gradio as gr

In [4]:
model_name = 'google/pegasus-large'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
abs_test_model =  PegasusForConditionalGeneration.from_pretrained(model_name) # we do not specify pretrained=True, i.e. do not load default weights
abs_test_model.load_state_dict(torch.load('/content/drive/MyDrive/models/abs_model_weights.pth'))

ext_test_model=torch.load('/content/drive/MyDrive/models/ext_model_weights.pth')

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [5]:
def text_summarizer(input_text,model):
  if model=='Abstractive':
    batch = tokenizer(input_text, truncation=True, padding='longest', return_tensors="pt")
    translated = abs_test_model.generate(**batch)
    abs_summary = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return abs_summary[0]
  elif model == 'Extractive':
    ext_summary = ''.join(ext_test_model(input_text,min_length=200))
    return ext_summary

In [ ]:
TextSummerizerUI = gr.Interface(
    fn=text_summarizer, 
    inputs=[
        gr.inputs.Textbox(lines=10, placeholder="Enter your Text!!"),
        gr.inputs.Dropdown(["Abstractive", "Extractive"]),
    ], 
    outputs='text', 
    title="Text Summarizer", 
    theme='dark')
TextSummerizerUI.launch(inbrowser=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://11491.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)
